In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.autograd import Variable

In [2]:
train_data_raw = pd.read_csv('./Dataset/sign_mnist_train.csv', sep=",")
test_data_raw = pd.read_csv('./Dataset/sign_mnist_test.csv', sep=",")

In [3]:
train_labels = train_data_raw['label']
train_data_raw.drop('label', axis=1, inplace=True) # dropping the label column

test_labels = test_data_raw['label']
test_data_raw.drop('label', axis=1, inplace=True) # dropping the label column

In [4]:
train_data = train_data_raw.values
train_labels = train_labels.values

test_data = test_data_raw.values
test_labels = test_labels.values

In [5]:
set(train_labels) #No  9=J or 25=Z because of gesture motions 

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24}

In [6]:
dim=28

In [7]:
pixels = train_data[20].reshape(dim, dim)
plt.subplot(221)
sns.heatmap(data=pixels)

pixels = train_data[30].reshape(dim, dim)
plt.subplot(222)
sns.heatmap(data=pixels)

ValueError: cannot reshape array of size 784 into shape (100,100)

In [ ]:
reshaped_train = []
for i in train_data:
    reshaped_train.append(i.reshape(1, dim, dim))
train_data = np.array(reshaped_train)

reshaped_test = []
for i in test_data:
    reshaped_test.append(i.reshape(1, dim, dim))
test_data = np.array(reshaped_train)

In [ ]:
train_x = torch.FloatTensor(train_data)
train_y = torch.LongTensor(train_labels.tolist())

test_x = torch.FloatTensor(train_data)
test_y = torch.LongTensor(train_labels.tolist())

In [ ]:
alph = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:"f", 6:'g', 7:'h', 8:'i', 9:'j', 10:'k', 11:'l', 12:'m', 13:'n',
        14:'o', 15:'p', 16:'q', 17:'r', 18:'s', 19:'t', 20:'u', 21:'v', 22:'w', 23:'x', 24:'y', 25:'z'}

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,20,3)
        self.conv3 = nn.Conv2d(20,30,3)
        
        self.pool = nn.MaxPool2d(2)
        self.dropout = nn.Dropout2d(0.2)
        
        self.fc1 = nn.Linear(30*3*3, 270)
        self.fc2 = nn.Linear(270,26)
        
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = x.view(-1, 30 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.softmax(F.relu(self.fc2(x)))
        
        return(x)
        
    def test(self, predictions,labels):
        self.eval()
        correct = 0
        for p,l in zip(predictions,labels):
            if p==l:
                correct+=1
        acc = correct/len(predictions)
        return(acc, correct, len(predictions))
    
    def evaluate(self, predictions,labels):
        correct = 0
        for p,l in zip(predictions,labels):
            if p==l:
                correct+=1
        acc = correct/len(predictions)
        return(acc)

In [ ]:
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    model = Network().cuda()

In [ ]:
summary(model, (1,dim,dim)) #takes the model and the input tensor shape, displays the output shape

In [ ]:
epochs = 50
batch_size = 100
learning_rate = 0.001

In [ ]:
optimizer = optim.SGD(model.parameters(), learning_rate, momentum=0.007)
criterion = nn.CrossEntropyLoss()

In [ ]:
# def train(epochs, model, optimizer, criterion, use_cuda):
#     loss_log = []
#     acc_log = []

#     for epoch in range(1, epochs+1):
#         train_loss = 0.0
        
#         model.train()
        
#         for index, data in enumerate(train_data):
#             target = train_labels[index]
            
#             if use_cuda:
#                 data.cuda()
# #                 target.cuda()
                
#             optimizer.zero_grad()
#             output = model(data)
#             loss = criterion(output, target)
#             loss.backward()
#             optimizer.step()
            
#             if index%1000==0:
#                 loss_log.append(loss.item())
#                 acc_log.append(net.evaluate(torch.max(net(Variable(test_x[:500])).data, 1)[1], test_y[:500]))
            
            
#             # print training/validation statistics 
#             print('Epoch: {} \tTraining Loss: {:.6f} '.format(
#                 epoch, 
#                 train_loss
#                 ))

            
#     # return trained model
#     return model

In [ ]:
# # train the model
# model_scratch = train(epochs, model, optimizer, criterion, use_cuda)

In [ ]:
loss_log = []
acc_log = []

for e in range(80):
    for i in range(0, train_x.shape[0], 100):
        x_mini = train_x[i:i + 100] 
        y_mini = train_y[i:i + 100] 
        
        if use_cuda:
            x_mini = x_mini.cuda()
            y_mini = y_mini.cuda()
            test_x = test_x.cuda()
            test_y = test_y.cuda()
        
        optimizer.zero_grad()
        net_out = model(Variable(x_mini))
        
        loss = criterion(net_out, Variable(y_mini))
        loss.backward()
        optimizer.step()
        
        if i % 1000 == 0:
            #pred = net(Variable(test_data_formated))
            loss_log.append(loss.item())
            acc_log.append(model.evaluate(torch.max(model(Variable(test_x[:500])).data, 1)[1], test_y[:500]))
        
    print('Epoch: {} - Loss: {:.6f}'.format(e + 1, loss.item()))

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(loss_log[2:])
plt.plot(acc_log)
plt.plot(np.ones(len(acc_log)), linestyle='dashed')
plt.show()

In [ ]:
predictions = model(Variable(test_x))
accuracy, correct, total = model.test(torch.max(predictions.data, 1)[1], test_y)

In [ ]:
print("Accuracy = " + str(accuracy) + " ("+str(correct)+"/"+str(total)+")")

In [ ]:
def predict(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (dim,dim))
    plt.imshow(img, cmap="gray")
    img = img.reshape(1,1, dim, dim)
    np_img = np.array(img)
    input_img = torch.FloatTensor(np_img)
    
    if use_cuda:
        input_img = input_img.cuda()
        
    pred = model(Variable(input_img))
    pred = torch.max(pred.data, 1)[1].cpu().numpy()[0]
    return pred

In [ ]:
prediction = predict('./c.jpg')
lab = 'c'

In [ ]:
print("Prediction: {}".format(alph[prediction]))
print("Actual Label: {}".format(lab))

In [ ]:
# pixels = cv2.imread('./c.jpg').reshape(28, 28)
# plt.subplot(223)
# sns.heatmap(data=pixels)
# lab = 'c'
# test_sample = torch.FloatTensor([pixels.reshape(1, 28, 28).tolist()])
# pred = model(Variable(input_img))
# print("Prediction: {}".format(alph[torch.max(net_out_sample.data, 1)[1].numpy()[0]]))
# print("Actual Label: {}".format(lab))
